# Keto/Vegan Diet classifier
You are presented with a large recipe database (allrecipes.com) - and you are asked to classify the recipes to keto (low carb) meals, and vegan (no animal products) meals.

In [1]:
from opensearchpy import OpenSearch
from decouple import config
import pandas as pd

client = OpenSearch(
    hosts=[config('OPENSEARCH_URL', 'http://localhost:9200')],
    http_auth=None,
    use_ssl=False,
    verify_certs=False,
    ssl_show_warn=False,
)

# Recipes Index
Our data is stored in OpenSearch, there are two ways to query it:
## Elasticsearch Syntax

In [2]:
query = {
    "query": {
        "match": {
            "description": { "query": "egg" }
        }
    }
}

res = client.search(
    index="recipes",
    body=query,
    size=2
)

hits = res['hits']['hits']
hits

[{'_index': 'recipes',
  '_id': 'TDKZP5cBWnLoii-wvQwm',
  '_score': 3.9819193,
  '_source': {'title': 'Genuine Egg Noodles',
   'description': 'These egg noodles are the original egg noodles.  ',
   'instructions': ['Combine flour, salt and baking powder. Mix in eggs and enough water to make the dough workable. Knead dough until stiff. Roll into ball and cut into quarters. Using 1/4 of the dough at a time, roll flat to about 1/8 inch use flour as needed, top and bottom, to prevent sticking. Peel up and roll from one end to the other. Cut roll into 3/8 inch strips. Noodles should be about 4 to 5 inches long depending on how thin it was originally flattened. Let dry for 1 to 3 hours.',
    'Cook like any pasta or, instead of drying first cook it fresh but make sure water is boiling and do not allow to stick. It takes practice to do this right.'],
   'ingredients': ['2 cups Durum wheat flour',
    '1/2 teaspoon salt',
    '1/4 teaspoon baking powder',
    '3 eggs',
    'water as needed'],

## SQL syntax

In [3]:
query = """
SELECT *
FROM recipes
WHERE description like '%egg%'
LIMIT 10
"""

res = client.sql.query(body={'query': query})
df = pd.DataFrame(res["datarows"], columns=[c["name"] for c in res["schema"]])
df

,description,ingredients,instructions,photo_url,title
0,These cookies are made without eggs.,"[1 cup butter, 1 cup white sugar, 1 tablespoon...",[In mixing bowl beat butter or margarine until...,http://images.media-allrecipes.com/userphotos/...,Coconut Rolled Sugar Cookies
1,"These are easy to make, using no eggs, or flou...","[1/2 cup butter, 1 cup crushed saltine cracker...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Easy Soda Cracker Bars
2,Have egg whites left over? This recipe will p...,"[2 egg whites, 1/2 teaspoon cream of tartar, 1...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/global/reci...,Chippers
3,A yummy dairy-free treat! You can use egg subs...,"[2/3 cup unbleached all-purpose flour, 2/3 cup...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Date-Nut and Brown Sugar Bars
4,A great eggless recipe.,"[1 cup white sugar, 1 cup shortening, 1 cup da...",[Preheat oven to 425 degrees F (220 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Eggless Ginger Cookies
5,Rolled sugar cookies painted with a egg yolk p...,"[1/2 cup butter, softened, 1/2 cup shortening,...",[Cream butter or margarine and shortening; gra...,http://images.media-allrecipes.com/global/reci...,Egg Yolk Painted Christmas Cookies
6,These may be frosted with an icing made from 2...,"[1 cup all-purpose flour, 1/3 cup white sugar,...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Walnut Cherry Delights
7,"A chip cookie with a difference! Light, crispy...","[1 cup margarine, 1 cup confectioners' sugar, ...",[Preheat oven to 300 degrees F (150 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Cherry Chip Cookies I
8,Polish cookies made with hard boiled egg yolks.,"[2 eggs, 1 cup butter, 3/4 cup white sugar, 1 ...",[Place eggs (and water to cover) in a medium s...,http://images.media-allrecipes.com/global/reci...,Polish Poppy Seed Cookies
9,Apple and Spice cookies - egg and milk free. ...,"[2 cups sifted all-purpose flour, 1 teaspoon b...",[Cream together shortening and brown sugar. St...,http://images.media-allrecipes.com/userphotos/...,Apple and Spice Cookies


# Task
Please write 2 classifiers, for vegan meals and for keto meals.

Unlike traditional machine learning tasks, the label is not available in dataset.

However, the definition of a vegan meal and a ketogenic meal is very clear and easily verifiable.

### For the purposes of this exercise:
* A **Vegan meal** cannot have even a single animal product in its ingredient list (eggs, milk, meat, etc).
* A **Keto meal** cannot have even a single ingredient with more than 10g of carbs per 100g portion (e.g. eggs are keto, while apples are not).
* A meal can be both vegan and keto (e.g. avocado) while most meals are neither.

## Example heuristic:

In [4]:
def is_ingredient_vegan(ing):
    for animal_product in "egg meat milk butter veel lamb beef chicken sausage".split():
        if animal_product in ing:
            return False
    return True

def is_vegan_example(ingredients):
    return all(map(is_ingredient_vegan, ingredients))
    
df["vegan"] = df["ingredients"].apply(is_vegan_example)
df

,description,ingredients,instructions,photo_url,title,vegan
0,These cookies are made without eggs.,"[1 cup butter, 1 cup white sugar, 1 tablespoon...",[In mixing bowl beat butter or margarine until...,http://images.media-allrecipes.com/userphotos/...,Coconut Rolled Sugar Cookies,False
1,"These are easy to make, using no eggs, or flou...","[1/2 cup butter, 1 cup crushed saltine cracker...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Easy Soda Cracker Bars,False
2,Have egg whites left over? This recipe will p...,"[2 egg whites, 1/2 teaspoon cream of tartar, 1...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/global/reci...,Chippers,False
3,A yummy dairy-free treat! You can use egg subs...,"[2/3 cup unbleached all-purpose flour, 2/3 cup...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Date-Nut and Brown Sugar Bars,False
4,A great eggless recipe.,"[1 cup white sugar, 1 cup shortening, 1 cup da...",[Preheat oven to 425 degrees F (220 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Eggless Ginger Cookies,True
5,Rolled sugar cookies painted with a egg yolk p...,"[1/2 cup butter, softened, 1/2 cup shortening,...",[Cream butter or margarine and shortening; gra...,http://images.media-allrecipes.com/global/reci...,Egg Yolk Painted Christmas Cookies,False
6,These may be frosted with an icing made from 2...,"[1 cup all-purpose flour, 1/3 cup white sugar,...",[Preheat oven to 350 degrees F (180 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Walnut Cherry Delights,False
7,"A chip cookie with a difference! Light, crispy...","[1 cup margarine, 1 cup confectioners' sugar, ...",[Preheat oven to 300 degrees F (150 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Cherry Chip Cookies I,True
8,Polish cookies made with hard boiled egg yolks.,"[2 eggs, 1 cup butter, 3/4 cup white sugar, 1 ...",[Place eggs (and water to cover) in a medium s...,http://images.media-allrecipes.com/global/reci...,Polish Poppy Seed Cookies,False
9,Apple and Spice cookies - egg and milk free. ...,"[2 cups sifted all-purpose flour, 1 teaspoon b...",[Cream together shortening and brown sugar. St...,http://images.media-allrecipes.com/userphotos/...,Apple and Spice Cookies,True


The heuristic above is very simplistic, with many false positives as it relies on key-word matching.

Common failure modes:
- "Peanut butter" would not be classified as vegan, because butter is assumed to be dairy
- "eggless" would not be classified as vegan, because "egg" is a prefix of "eggless"
- "pork" and "bacon" would be classified as vegan, because we did not explicitly list them as animal products.


# Submission
## 1. Implement Diet Classifiers
Implement two functions in `diet_classifiers.py` in this repo - and make sure the flask server shows the "keto" and "vegan" badges next to the appropriate recipes.

> **Note**
>
> This repo contains two `diet_classifiers.py` files:
> 1. One in this folder (`nb/src/diet_classifiers.py`)
> 2. One in the Flask web app folder (`web/src/diet_classifiers.py`)
>
> You can develop your solution here in the notebook environment, but to apply your solution 
> to the Flask app you will need to copy your implementation into the `diet_classifiers.py` 
> file in the Flask folder!!!

In [5]:
def is_ingredient_keto(ingredient):
    # TODO: complete
    return False

def is_ingredient_vegan(ingredient):
    # TODO: complete
    return False    

For your convenience, you can sanity check your solution on a subset of labeled recipes by running `diet_classifiers.py`

In [7]:
! python diet_classifiers.py --ground_truth /usr/src/data/ground_truth_sample.csv

===Keto===
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
              precision    recall  f1-

## 2. Repository Setup
Create a new **private** github repo and invite `argmax2025` to collaborate with you, **do not** share your code on a **forked** repo.

## 3. Application Form
Fill in the [US Application Form](https://forms.clickup.com/25655193/f/rexwt-1832/L0YE9OKG2FQIC3AYRR) or the [IL Application Form](https://forms.clickup.com/25655193/f/rexwt-1812/IP26WXR9X4P6I4LGQ6)

## Evaluation process
Your submission will be evaluated based on the following criteria:
1. **Readabilty and logic** - make sure you explain your model, and reasoning behind it. If you used external source, make sure you include them.
2. **Execuable** - Your code would be run "as is" once pulled from github, make sure the syntax is correct and all of the pathes are relative.
3. **Accuracy** - We have created a holdout set of 20k recipes, with their proper classification. Your submission would be compared to the ground truth data.
## Next steps
On the follow-up interview would include a 3 hour live-coding session, you will be requested to extend your submission in various ways.
Make sure you are at a quiet setting, with python workstation capable of running your submission.